# Conexiones con Ollama mediante su API

## En esta Notebook se muestra la forma de obtener conexión con modelos LLM corriendo de manera local.

Documentación: https://github.com/ollama/ollama/blob/main/docs/api.md

## Importamos las librerías.
- Request para hacer peticiones a una API
- dotenv para trabajar con archivos .env
- os para poder obtener las variables de entorno

In [12]:
import requests
from dotenv import load_dotenv
import os
load_dotenv()

URL_OLLAMA = os.getenv("URL_OLLAMA")

## Hacer un pull de un modelo.

Lista de modelos disponible: https://ollama.com/search

In [13]:
def pull(model):
    #URL por defecto para descargas de modelos LLM
    url = f'{URL_OLLAMA}/api/pull'
    # Comprobar que los valores obligatorios están presentes
    response_data = {
        "model": model,
        "stream": False #Evita usar streaming. Que no mande la información en partes y la mande en su lugar todo en conjunto
    }
    try:
        #Usa POST para enviar los datos al servidor
        response = requests.post(url, json=response_data)
        if response.status_code == 200:
            return response.json(), 200
        else:
            print("Error:", response.status_code, response.text)
            return {"error": f"Error {response.status_code}: {response.text}"}, response.status_code
    except requests.exceptions.RequestException as e:
        print("An error occurred:", e)
        return {"error": f"An error occurred: {e}"}, 500

In [14]:
model_pull = "gemma3:4b"

response = pull(model_pull)
print(response)

({'status': 'success'}, 200)


## Ver los modelos disponibles

In [15]:
url_tags = f"{URL_OLLAMA}/api/tags"
#Usa GET para obtener información de todos los modelos locales
response = requests.get(url_tags)
if response.status_code == 200:
    tags = response.json()
    print(tags)
    print("Modelos:")
    modelos = tags.get("models", [])
    for modelo in modelos:
        print(modelo['name'])
        print("--")
else:
    print("Error al obtener los modelos:", response.status_code, response.text)

{'models': [{'name': 'gemma3:4b', 'model': 'gemma3:4b', 'modified_at': '2025-06-19T21:57:02.3379454Z', 'size': 3338801804, 'digest': 'a2af6cc3eb7fa8be8504abaf9b04e88f17a119ec3f04a3addf55f92841195f5a', 'details': {'parent_model': '', 'format': 'gguf', 'family': 'gemma3', 'families': ['gemma3'], 'parameter_size': '4.3B', 'quantization_level': 'Q4_K_M'}}, {'name': 'llama3.2:latest', 'model': 'llama3.2:latest', 'modified_at': '2025-06-19T21:15:35.4544573Z', 'size': 2019393189, 'digest': 'a80c4f17acd55265feec403c7aef86be0c25983ab279d83f3bcd3abbcb5b8b72', 'details': {'parent_model': '', 'format': 'gguf', 'family': 'llama', 'families': ['llama'], 'parameter_size': '3.2B', 'quantization_level': 'Q4_K_M'}}]}
Modelos:
gemma3:4b
--
llama3.2:latest
--


## Uso de generación de texto mediante LLM

- Model es el modelo a utilizar (Obligatorio)
- Prompt es el texto que se le envía al modelo para generar una respuesta (opcional. Puede requerir solo activar el modelo sin prompt)
- Format es el formato de salida que se desee (opcional) - Usa "json" o un JSON Schema
- Keep_alive permite hacer que el modelo perdure en memoria los minutos que se requiera. 0 minutos libera memoria y -1 minutos lo hace para siempre (opcional)

In [ ]:
def generate(model,prompt = None, format = None, keep_alive = None):
    url = f'{URL_OLLAMA}/api/generate'
    # Comprobar que los valores obligatorios están presentes
    response_data = {
        "model": model,
        "stream": False #Evita usar streaming. Que no mande la información en partes y la mande en su lugar todo en conjunto
    }
    #Corroborar los valores opcionales y los añade en dado caso de existir
    if prompt:
        response_data["prompt"] = prompt
    if format:
        response_data["format"] = format
    if keep_alive:
        response_data["keep_alive"] = keep_alive
    try:
        response = requests.post(url, json=response_data)
        if response.status_code == 200:
            return response.json()['response'], 200
        else:
            print("Error:", response.status_code, response.text)
            return {"error": f"Error {response.status_code}: {response.text}"}, response.status_code
    except requests.exceptions.RequestException as e:
        print("An error occurred:", e)
        return {"error": f"An error occurred: {e}"}, 500

### Uso para completamiento de texto

In [17]:
prompt = "What is the capital of France?"
model = "llama3.2"

response = generate(model, prompt)
print("Respuesta:",response[0])
print("Codigo de respuesta:", response[1])

Respuesta: The capital of France is Paris.
Codigo de respuesta: 200


### Uso para completamiento de texto con rúbrica

**Nota**: Para el apartado format Utiliza JSON schema para eso.
Links: 
- https://tour.json-schema.org/
- https://json-schema.org/


In [18]:
prompt = "Ollama is 22 years old and is busy saving the world. Respond using JSON"
model = "llama3.2"
format = {
    "type": "object",
    "properties": {
      "age": {
        "type": "integer"
      },
      "available": {
        "type": "boolean"
      }
    },
    "required": [
      "age",
      "available"
    ]
  }

response = generate(model, prompt, format)
print("Respuesta:",response[0])
print("Codigo de respuesta:", response[1])

Respuesta: { "age": 22, "available" : true }
Codigo de respuesta: 200
